In [30]:
import numpy as np
from bonebox.phantoms.TrabeculaeVoronoi_Shi import *
import ipyvolume as ipv
# import nibabel as nib
import random
import math
import os
from tqdm import tqdm

#####################################################################
folderName = 'BoneSet'

# Parameters for generating phantom mesh
# Volume size in mm
Sxyz                = (5,5,5)
# Volume dim in Voxels
volumeSizeVoxels    = (400,400,400)
# Spacing of seed points
Spacingxyz          = (1, 1, 1)
# Distubance radius in mm
Rxyz                = 0.0

# For random number gneration
randState           = 123 # for repeatability
# randState           = random.choice(range(100))

# Edge and Face fraction to retain
edgesRetainFraction = 0.5
facesRetainFraction = 0.3
# Dilation radius in voxels
dilationRadius      = 3

# Voxel Size
voxelSize = np.array(Sxyz) / np.array(volumeSizeVoxels)
#####################################################################

if not os.path.exists(folderName):
    os.mkdir(folderName)

In [31]:
# Generate Seed Points
points = makeSeedPointsCartesian_LineorArc(Spacingxyz[0], \
                                           Spacingxyz[1], \
                                           Spacingxyz[2], \
                                           10, 0, 0, \
                                           10, 0, 0, \
                                           Sxyz,7)

# Random perturbation
ppoints = perturbSeedPointsCartesianUniformXYZ(points, Rxyz, randState=randState)

# Generate voronoi tessilation
vor, ind = applyVoronoi(ppoints, Sxyz)

In [32]:
fig = ipv.figure()
scatter = ipv.scatter(ppoints[:,0], ppoints[:,1], ppoints[:,2])
ipv.show()
ipv.pylab.view(105,20,2.5)

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

(105, 20, 2.5)

In [33]:
# Finalize initial edges and faces
uniqueEdges, uniqueFaces = findUniqueEdgesAndFaces(vor, ind,Sxyz)

# Remove Faces
uniqueFacesRetain, facesRetainInd = filterFacesRandomUniform(uniqueFaces, 
                                                             facesRetainFraction, 
                                                             randState=randState)
print("Face number final = %d" % len(facesRetainInd))

# Remove Edge that is part of a face
for j in tqdm(range(len(uniqueFacesRetain))):
    temp = [np.sum(np.isin(uniqueEdges[x], uniqueFacesRetain[j]).astype(int)) == 2 for x in range(len(uniqueEdges))]
    temp = np.array(temp).astype(int)
    if j == 0:
        t = temp
    else:
        t = t + temp
edge = np.delete(uniqueEdges,np.where(t>0)[0].astype(int), axis=0)
print("Edge number = %d" % len(edge))

# Randomly Remove Edge
uniqueEdgesRetain, edgesRetainInd = filterEdgesRandomUniform(edge, 
                                                             edgesRetainFraction, 
                                                             randState=randState)
print("Edge number final = %d\n" % len(edgesRetainInd))

# Rod to plate ratio
RodToPlate = len(edge)/len(facesRetainInd)
print("Max Rod/plate number = %f" % RodToPlate)
RodToPlate = len(edgesRetainInd)/len(facesRetainInd)
print("Final Rod/plate number = %f" % RodToPlate)


# Generate Volume base on the remaining face and edges
volumeEdges = makeSkeletonVolumeEdges(vor.vertices, 
                                      uniqueEdgesRetain, 
                                      voxelSize, 
                                      volumeSizeVoxels)
volumeFaces = makeSkeletonVolumeFaces(vor.vertices, 
                                      uniqueFacesRetain, 
                                      voxelSize, 
                                      volumeSizeVoxels)


volumeDilated = dilateVolumeSphereUniform(np.logical_or(volumeEdges,volumeFaces), \
                                          dilationRadius)

volumeDilated = volumeDilated * 1;

#####################################################################################

21
21
Face number final = 80


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 93.16it/s]


Edge number = 218
Edge number final = 109

Max Rod/plate number = 2.725000
Final Rod/plate number = 1.362500


In [35]:
fig = ipv.figure()

indices = np.nonzero(volumeEdges)
scatter = ipv.scatter(indices[0]/1, indices[1]/1, indices[2]/1, color='blue', marker='box',  size=1)

indices = np.nonzero(volumeFaces)
scatter = ipv.scatter(indices[0]/1, indices[1]/1, indices[2]/1, color='red', marker='box',  size=1)

ipv.show()
ipv.pylab.view(105,20,2.5)

fig = ipv.figure()

indices = np.nonzero(volumeDilated)
scatter = ipv.scatter(indices[0]/1, indices[1]/1, indices[2]/1, color='red', marker='box',  size=1)

ipv.show()
ipv.pylab.view(105,20,2.5)

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

(105, 20, 2.5)